In [20]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters



In [21]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [22]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\guyv2\\OneDrive\\שולחן העבודה\\hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [23]:
a_turns = pd.read_csv('../data/a_t_turns.csv')
p_turns = pd.read_csv('../data/p_t_turns.csv')
b_turns = pd.read_csv('../data/b_t_turns.csv')

In [32]:
a_turns

,game_id,turn_number,points,score,is_player,turn_type_Challenge,turn_type_End,turn_type_Exchange,turn_type_Pass,turn_type_Play,...,Z,.,TileType.N,TileType.L2,TileType.L3,TileType.W2,TileType.W3,move_len,jokers_num,special_tile
0,1,1,10,10,0,0,0,0,0,1,...,0,0,3,0,0,0,0,3,0,0
1,1,2,18,18,1,0,0,0,0,1,...,0,0,2,1,0,0,0,3,0,1
2,1,3,16,26,0,0,0,0,0,1,...,0,0,3,0,1,0,0,4,0,1
3,1,4,16,34,1,0,0,0,0,1,...,0,0,1,0,0,1,0,2,0,1
4,1,5,28,54,0,0,0,0,0,1,...,0,2,6,1,0,1,0,8,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005493,72773,22,18,376,1,0,0,0,0,1,...,0,0,1,0,0,0,1,2,0,1
2005494,72773,23,24,331,0,0,0,0,0,1,...,0,1,2,1,0,0,1,4,0,2
2005495,72773,24,7,383,1,0,0,0,0,1,...,0,1,3,0,0,0,1,4,0,1
2005496,72773,25,11,342,0,0,0,0,0,1,...,0,0,2,1,0,0,0,3,0,1


In [26]:
settings = MinimalFCParameters()
a_extracted_features = extract_features(a_turns, column_id="game_id", column_sort="turn_number", n_jobs=10, default_fc_parameters=settings)
p_extracted_features = extract_features(p_turns, column_id="game_id", column_sort="p_turn_number", n_jobs=10, default_fc_parameters=settings)
b_extracted_features = extract_features(b_turns, column_id="game_id", column_sort="b_turn_number", n_jobs=10, default_fc_parameters=settings)

Feature Extraction: 100%|██████████| 50/50 [13:08<00:00, 15.77s/it]


In [30]:
p_extracted_features

,p_points__sum_values,p_points__median,p_points__mean,p_points__length,p_points__standard_deviation,p_points__variance,p_points__root_mean_square,p_points__maximum,p_points__absolute_maximum,p_points__minimum,...,p_special_tile__sum_values,p_special_tile__median,p_special_tile__mean,p_special_tile__length,p_special_tile__standard_deviation,p_special_tile__variance,p_special_tile__root_mean_square,p_special_tile__maximum,p_special_tile__absolute_maximum,p_special_tile__minimum
1,429.0,25.0,30.642857,14.0,24.563641,603.372449,39.272855,98.0,98.0,2.0,...,15.0,1.0,1.071429,14.0,0.593330,0.352041,1.224745,2.0,2.0,0.0
2,488.0,29.0,34.857143,14.0,29.743804,884.693878,45.822639,94.0,94.0,0.0,...,16.0,1.0,1.142857,14.0,0.832993,0.693878,1.414214,3.0,3.0,0.0
3,440.0,28.0,31.428571,14.0,20.900151,436.816327,37.743495,103.0,103.0,13.0,...,17.0,1.0,1.214286,14.0,0.557875,0.311224,1.336306,2.0,2.0,0.0
4,119.0,5.5,8.500000,14.0,8.287254,68.678571,11.871334,26.0,26.0,0.0,...,12.0,1.0,0.857143,14.0,0.515079,0.265306,1.000000,2.0,2.0,0.0
5,325.0,20.5,20.312500,16.0,12.050512,145.214844,23.618055,51.0,51.0,0.0,...,16.0,1.0,1.000000,16.0,0.612372,0.375000,1.172604,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72769,384.0,19.0,21.333333,18.0,36.525486,1334.111111,42.299199,82.0,82.0,-78.0,...,16.0,1.0,0.888889,18.0,0.657342,0.432099,1.105542,2.0,2.0,0.0
72770,97.0,6.0,6.062500,16.0,6.407698,41.058594,8.821139,24.0,24.0,-10.0,...,15.0,1.0,0.937500,16.0,0.555512,0.308594,1.089725,2.0,2.0,0.0
72771,390.0,20.5,24.375000,16.0,18.600655,345.984375,30.661458,83.0,83.0,4.0,...,18.0,1.0,1.125000,16.0,0.695971,0.484375,1.322876,2.0,2.0,0.0
72772,388.0,19.0,24.250000,16.0,18.723314,350.562500,30.636987,67.0,67.0,4.0,...,16.0,1.0,1.000000,16.0,0.500000,0.250000,1.118034,2.0,2.0,0.0


In [35]:
a_extracted_features.to_csv('../data/a_new_extracted_features_turns.csv', index_label='game_id')
p_extracted_features.to_csv('../data/p_new_extracted_features_turns.csv', index_label='game_id')
b_extracted_features.to_csv('../data/b_new_extracted_features_turns.csv', index_label='game_id')


In [4]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
a_turns = pd.read_csv('../data/a_new_extracted_features_turns.csv', index_col='game_id')
p_turns = pd.read_csv('../data/p_new_extracted_features_turns.csv', index_col='game_id')
b_turns = pd.read_csv('../data/b_new_extracted_features_turns.csv', index_col='game_id')

In [9]:
p_turns

,p_points__sum_values,p_points__median,p_points__mean,p_points__length,p_points__standard_deviation,p_points__variance,p_points__root_mean_square,p_points__maximum,p_points__absolute_maximum,p_points__minimum,...,p_jokers_num__sum_values,p_jokers_num__median,p_jokers_num__mean,p_jokers_num__length,p_jokers_num__standard_deviation,p_jokers_num__variance,p_jokers_num__root_mean_square,p_jokers_num__maximum,p_jokers_num__absolute_maximum,p_jokers_num__minimum
game_id,,,,,,,,,,,,,,,,,,,,,
1,429.0,25.0,30.642857,14.0,24.563641,603.372449,39.272855,98.0,98.0,2.0,...,2.0,0.0,0.142857,14.0,0.515079,0.265306,0.534522,2.0,2.0,0.0
2,488.0,29.0,34.857143,14.0,29.743804,884.693878,45.822639,94.0,94.0,0.0,...,1.0,0.0,0.071429,14.0,0.257539,0.066327,0.267261,1.0,1.0,0.0
3,440.0,28.0,31.428571,14.0,20.900151,436.816327,37.743495,103.0,103.0,13.0,...,1.0,0.0,0.071429,14.0,0.257539,0.066327,0.267261,1.0,1.0,0.0
4,119.0,5.5,8.500000,14.0,8.287254,68.678571,11.871334,26.0,26.0,0.0,...,0.0,0.0,0.000000,14.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,325.0,20.5,20.312500,16.0,12.050512,145.214844,23.618055,51.0,51.0,0.0,...,0.0,0.0,0.000000,16.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72769,384.0,19.0,21.333333,18.0,36.525486,1334.111111,42.299199,82.0,82.0,-78.0,...,2.0,0.0,0.111111,18.0,0.314270,0.098765,0.333333,1.0,1.0,0.0
72770,97.0,6.0,6.062500,16.0,6.407698,41.058594,8.821139,24.0,24.0,-10.0,...,1.0,0.0,0.062500,16.0,0.242061,0.058594,0.250000,1.0,1.0,0.0
72771,390.0,20.5,24.375000,16.0,18.600655,345.984375,30.661458,83.0,83.0,4.0,...,0.0,0.0,0.000000,16.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [10]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([('get_relavent_turns', SelectRowsTransformer(train.index.unique(), target=T_NAME)),
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [15]:
preprocessor = PreprocessorBuilder(games, G_NAME, a_turns, T_NAME, prePipe).build()
n_games, a_n_turns, n_ratings = preprocessor.process(train, DATA_NAME)
p_n_turns = p_turns.loc[train.index.unique(),:]
b_n_turns = b_turns.loc[train.index.unique(),:]



In [18]:
a_n_turns

,points__sum_values,points__median,points__mean,points__length,points__standard_deviation,points__variance,points__root_mean_square,points__maximum,points__absolute_maximum,points__minimum,...,jokers_num__sum_values,jokers_num__median,jokers_num__mean,jokers_num__length,jokers_num__standard_deviation,jokers_num__variance,jokers_num__root_mean_square,jokers_num__maximum,jokers_num__absolute_maximum,jokers_num__minimum
game_id,,,,,,,,,,,,,,,,,,,,,
1,764.0,24.0,28.296296,27.0,20.956418,439.171468,35.211530,98.0,98.0,2.0,...,2.0,0.0,0.074074,27.0,0.377705,0.142661,0.384900,2.0,2.0,0.0
3,758.0,25.0,28.074074,27.0,19.584434,383.550069,34.230158,103.0,103.0,0.0,...,2.0,0.0,0.074074,27.0,0.261891,0.068587,0.272166,1.0,1.0,0.0
4,597.0,16.0,20.586207,29.0,19.526317,381.277051,28.373737,74.0,74.0,0.0,...,1.0,0.0,0.034483,29.0,0.182466,0.033294,0.185695,1.0,1.0,0.0
5,752.0,21.0,24.258065,31.0,16.688740,278.514048,29.444316,81.0,81.0,0.0,...,2.0,0.0,0.064516,31.0,0.245670,0.060354,0.254000,1.0,1.0,0.0
6,805.0,28.0,32.200000,25.0,19.720041,388.880000,37.758708,74.0,74.0,0.0,...,2.0,0.0,0.080000,25.0,0.271293,0.073600,0.282843,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,754.0,20.0,23.562500,32.0,18.496516,342.121094,29.955175,85.0,85.0,-10.0,...,2.0,0.0,0.062500,32.0,0.242061,0.058594,0.250000,1.0,1.0,0.0
72770,687.0,11.5,21.468750,32.0,24.590375,604.686523,32.643433,106.0,106.0,-10.0,...,2.0,0.0,0.062500,32.0,0.242061,0.058594,0.250000,1.0,1.0,0.0
72771,783.0,20.0,25.258065,31.0,21.527129,463.417274,33.187153,93.0,93.0,0.0,...,2.0,0.0,0.064516,31.0,0.245670,0.060354,0.254000,1.0,1.0,0.0


In [19]:
impute(a_n_turns)
a_features_filtered = select_features(a_n_turns, n_ratings, n_jobs=7)
a_features_filtered.to_csv('../data/a_new_features_filtered.csv', index_label='game_id')

KeyboardInterrupt: 

In [ ]:
a_features_filtered

,.__mean,score__absolute_maximum,.__sum_values,.__median,.__root_mean_square,points__root_mean_square,score__maximum,TileType.N__sum_values,points__median,points__sum_values,points__mean,score__standard_deviation,score__variance,score__root_mean_square,points__variance,points__standard_deviation,turn_type_Play__sum_values,score__mean,C__mean,P__mean,C__length,b__length,Q__length,R__length,k__length,S__length,B__length,a__length,n__length,g__length,T__length,U__length,turn_type_nan__length,V__length,z__length,y__length,turn_type_Timeout__length,P__length,l__length,O__length,...,I__median,A__sum_values,j__variance,j__standard_deviation,j__root_mean_square,j__mean,d__standard_deviation,d__root_mean_square,d__mean,d__variance,N__maximum,N__absolute_maximum,G__absolute_maximum,G__maximum,h__maximum,h__absolute_maximum,h__sum_values,z__sum_values,z__absolute_maximum,z__maximum,z__standard_deviation,z__root_mean_square,z__mean,z__variance,h__standard_deviation,h__variance,h__mean,h__root_mean_square,TileType.W2__root_mean_square,W__maximum,W__absolute_maximum,T__sum_values,B__absolute_maximum,B__maximum,C__maximum,C__absolute_maximum,H__maximum,H__absolute_maximum,A__median,turn_type_Pass__median
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.444444,429.0,12.0,0.0,0.816497,35.211530,429.0,78.0,24.0,764.0,28.296296,133.425411,17802.340192,257.931803,439.171468,20.956418,26.0,220.740741,0.074074,0.074074,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.666667,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,0.481481,440.0,13.0,0.0,0.793492,34.230158,440.0,74.0,25.0,758.0,28.074074,130.136691,16935.558299,234.282672,383.550069,19.584434,25.0,194.814815,0.074074,0.074074,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.693889,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1.137931,478.0,33.0,1.0,1.790781,28.373737,478.0,73.0,16.0,597.0,20.586207,121.426323,14744.351962,200.310104,381.277051,19.526317,27.0,159.310345,0.068966,0.034483,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,29.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.669534,1.0,1.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
5,0.451613,427.0,14.0,0.0,0.762001,29.444316,427.0,74.0,21.0,752.0,24.258065,108.961773,11872.668054,249.182729,278.514048,16.688740,29.0,224.096774,0.064516,0.064516,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.567962,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
6,0.600000,427.0,15.0,0.0,1.000000,37.758708,427.0,78.0,28.0,805.0,32.200000,122.291547,14955.222400,240.631752,388.880000,19.720041,23.0,207.240000,0.080000,0.080000,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.600000,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,0.781250,414.0,25.0,1.0,1.237437,29.955175,414.0,85.0,20.0,754.0,23.562500,124.978182,15619.5

In [ ]:
impute(p_n_turns)
p_features_filtered = select_features(p_n_turns, n_ratings, n_jobs=7)
p_features_filtered.to_csv('../data/p_new_features_filtered.csv', index_label='game_id')

In [ ]:
impute(a_n_turns)
b_features_filtered = select_features(b_n_turns, n_ratings, n_jobs=7)
b_features_filtered.to_csv('../data/b_new_features_filtered.csv', index_label='game_id')